In [1]:
#2018 HMDA Edit Testing File Generator
from collections import OrderedDict
from io import StringIO
import json
import os
import pandas as pd
import random

#custom imports
import lar_constraints
import lar_generator
from rules_engine import rules_engine


In [2]:
#2018 Filing Instruction Guide: https://www.consumerfinance.gov/data-research/hmda/static/for-filers/2018/2018-HMDA-FIG.pdf

use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
             'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependancies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
                    header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)

In [3]:
#load schemas for LAR and transmittal sheet
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))

In [4]:
lar_gen = lar_generator.lar_gen(lar_schema_df, ts_schema_df, counties=counties, tracts=tracts) #instantiate generator
lar_const = lar_constraints.lar_constraints(counties=counties, tracts=tracts)#instantiate constraints

In [5]:
#run constraint functions on row
file_length = 10 #set number of rows in test file
first = True
lei = None

def get_const_list():
    """Creates a list of constraints from the functions in the lar_constraints object."""
    constraints = [] 
    for func in dir(lar_const):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            constraints.append(func)
    return constraints
            
def apply_constraints(row=None, constraints=[]):
    """Applies all constraints in the constrains list and returns a LAR row in dictionary format."""
    for func in constraints: #loop over all constraints
            row = getattr(lar_const, func)(row) #apply constraint to row
    return row

In [6]:


for i in range(0, file_length): #loop over file length
    print("making new row\n\n")
    if lei:
        row = lar_gen.make_row(lei=lei)
    else:
        row = lar_gen.make_row() #create new row
    lei = row["lei"]
    diff = [1] #initialize diff for loop
    iters = 1
#diff needs a secondary check to 
    while len(diff) > 0:
        row_base = row.copy() #copy row to enable diff
        #apply constraint functions to LAR row
        print("\nstarting constraints iteration {iter}".format(iter=iters))
        
        row = apply_constraints(row, get_const_list())
        
        #convert initial and copied rows to sets for diff    
        initial_row = set(row_base.items())
        changed_row = set(row.items())
        diff = (initial_row - changed_row) #subtract row sets to show changes from constraint funcs
#            print(len(initial_row), len(changed_row)) #check number of fields, should be 110
        #show readout of number of changes made to new LAR row
        #print("changes from {func}: {vals}".format(func=str(func), vals=len(diff)))
        #print(row["app_date"], row["action_taken"])
        print(len(diff), "difference")
        iters+=1
    #create first row of dataframe
    if first:
        lar_frame = pd.DataFrame(row, index=[1])
        first = False
        print("finished row\n",lar_frame.columns)
    #add additional rows to dataframe
    else:
        #print("concating")
        print("finished row\n")
        new_lar = pd.DataFrame(row, index=[1])
        lar_frame = pd.concat([lar_frame, new_lar], axis=0)
lar_frame.reset_index(inplace=True) #reset index
lar_frame.drop("index", inplace=True, axis=1) #drop additional index column
print(iters)

making new row



starting constraints iteration 1
43 difference

starting constraints iteration 2
11 difference

starting constraints iteration 3
0 difference
finished row
 Index(['record_id', 'lei', 'uli', 'app_date', 'loan_type', 'loan_purpose',
       'preapproval', 'const_method', 'occ_type', 'loan_amount',
       ...
       'aus_code_5', 'aus_result_1', 'aus_result_2', 'aus_result_3',
       'aus_result_4', 'aus_result_5', 'aus_code_16', 'reverse_mortgage',
       'open_end_credit', 'business_purpose'],
      dtype='object', length=110)
making new row



starting constraints iteration 1
40 difference

starting constraints iteration 2
8 difference

starting constraints iteration 3
1 difference

starting constraints iteration 4
0 difference
finished row

making new row



starting constraints iteration 1
34 difference

starting constraints iteration 2
27 difference

starting constraints iteration 3
0 difference
finished row

making new row



starting constraints iteration 1
46 dif

In [7]:
#Quality and Macro field interrelationship constraints:

In [8]:
#Create a sample TS row
#Note: this will need to be more robust to include other federal agencies
# mlo_id needs NA option
#set dummy values for TS row
ts_row_small = OrderedDict()
ts_row_small["record_id"]="1"
ts_row_small["inst_name"]="Ficus Bank"
ts_row_small["calendar_year"]=str(2018)
ts_row_small["calendar_quarter"]="4"
ts_row_small["contact_name"]="Mr. Smug Pockets"
ts_row_small["contact_tel"]="555-555-5555"
ts_row_small["contact_email"]="pockets@ficus.com"
ts_row_small["contact_street_address"]="1234 Ficus Lane"
ts_row_small["office_city"]="Ficusville"
ts_row_small["office_state"]="UT"
ts_row_small["office_zip"]="84096"
ts_row_small["federal_agency"]="9"
ts_row_small["lar_entries"]= str(len(lar_frame))
ts_row_small["tax_id"]="01-0123456"
ts_row_small["lei"]=lar_frame.get_value(0, "lei")

In [9]:
#join LAR and TS rows to make an output file
def write_file(ts_input=None, lar_input=None, directory="../edits_files/", name="passes_all.txt"):
    """Takes a TS row as a dictionary and LAR data as a dataframe. Writes LAR data to file and 
    re-reads it to combine with TS data to make a full file."""
    #make directories for files if they do not exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    #write LAR dataframe to file
    parts_dir = directory+"file_parts/"
    if not os.path.exists(parts_dir):
        os.makedirs(parts_dir)
        
    lar_input.to_csv(parts_dir + "lar_data.txt", sep="|", header=False, index=False, index_label=False)
    #load LAR data as file rows
    with open(parts_dir + "lar_data.txt", 'r') as lar_data:
        lar = lar_data.readlines()

    with open(directory + name, 'w') as final_file:
        final_file.write("|".join(ts_input.values())+"\n")
        for line in lar:
            final_file.write("{line}".format(line=line))

In [10]:
#modify file for testing


In [11]:
#write sample file to disk
write_file(ts_input=ts_row_small, lar_input=lar_frame) #writes created file to disk
#validator engine uses the default: path="../edits_files/", data_file="passes_all.txt" for data files
validator = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties) #instantiate edits rules engine

In [12]:
#split TS and LAR using validator function
#validator creates class objects of each of these internally as well
ts_df, lar_df = validator.split_ts_row(path="../edits_files/", data_file="passes_all.txt")


In [13]:
for func in dir(validator):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        #print("applying:", func)
        getattr(validator, func)()


In [14]:
validator.results

[{'edit_name': 's300_1',
  'fields': 'record_id',
  'row_type': 'TS',
  'status': 'passed'},
 {'edit_name': 's300_2',
  'fields': 'record_id',
  'row_type': 'LAR',
  'status': 'passed'},
 {'edit_name': 's301', 'fields': 'LEI', 'row_type': 'TS', 'status': 'passed'},
 {'edit_name': 's302',
  'fields': 'calendar_year',
  'row_type': 'TS',
  'status': 'passed'},
 {'edit_name': 's304',
  'fields': '',
  'row_type': 'TS/LAR',
  'status': {'lar_entries': 'passed'}},
 {'edit_name': 's305', 'fields': 'all', 'row_type': 'LAR', 'status': 'passed'},
 {'edit_name': 'v600', 'fields': 'LEI', 'row_type': 'LAR', 'status': 'passed'},
 {'edit_name': 'v601_1',
  'fields': 'inst_name',
  'row_type': 'TS',
  'status': 'passed'},
 {'edit_name': 'v601_2',
  'fields': 'contact_name',
  'row_type': 'TS',
  'status': 'passed'},
 {'edit_name': 'v601_3',
  'fields': 'contact_email',
  'row_type': 'TS',
  'status': 'passed'},
 {'edit_name': 'v601_4',
  'fields': 'contact_street_address',
  'row_type': 'TS',
  'stat

In [15]:
ulis=[]
validator.lar_df[["loan_purpose", "preapproval","action_taken","app_race_1", "app_race_2", "app_race_3", "app_race_4", "app_race_5"
                  ,"uli"]][validator.lar_df.uli.isin(ulis)]

,loan_purpose,preapproval,action_taken,app_race_1,app_race_2,app_race_3,app_race_4,app_race_5,uli


In [16]:
results_df = pd.DataFrame(validator.results)

In [17]:
results_df[results_df.status=="failed"]

,edit_name,fail_count,fields,row_ids,row_type,status
214,v695,1.0,NMLS ID,[86FQDGA5KMKOTG4N4UAS1QTFTF7GUCRAH623H02NMEV19],LAR,failed


In [18]:
validator.lar_df[["action_taken", "preapproval", "interest_rate","mlo_id","uli"]]

,action_taken,preapproval,interest_rate,mlo_id,uli
0,2,2,15.15,VXP9JV69,86FQDGA5KMKOTG4N4UAS1TS4KV5BJYLYHV2A5DJEC4V76
1,2,2,4.04,S16SXENWQIT,86FQDGA5KMKOTG4N4UAS2985NTI0MXDQVW3SFIBKPIE15
2,6,2,13.13,TI05JCBW4SGS8NMVX,86FQDGA5KMKOTG4N4UASXK0WZLYUFCP5SS52I7Z862742
3,6,2,NA,,86FQDGA5KMKOTG4N4UAS1QTFTF7GUCRAH623H02NMEV19
4,1,2,15.15,1KIYCW,86FQDGA5KMKOTG4N4UASFP9ZOJH120XMFA3MQVWGN3D63
5,2,2,21.21,AUW8X2WT3PTFTZ9HM72,86FQDGA5KMKOTG4N4UASJAEV4F7NE77PTC343X09A1W95
6,4,2,NA,FQAVWV4EXB90UW0YXAP,86FQDGA5KMKOTG4N4UASME9P0ZTLKB4ZJQ86QEY9JOS55
7,6,2,14.14,ND41T,86FQDGA5KMKOTG4N4UASF3T28KNKP9J5K1FDTO45TPP60
8,6,2,NA,68J2IDJPWITM9,86FQDGA5KMKOTG4N4UAS6EMY1HF1N6ICKX7QB1PGD9I83
9,4,2,NA,TSVV1QECF,86FQDGA5KMKOTG4N4UASO31U2L2FP726D6BH94B6Z2W31


In [19]:
validator.lar_df.mlo_id = "d23lkj"

In [22]:
validator.lar_df.to_csv("../edits_files/passes_10_rows.txt", index=False, sep="|")